In [2]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# sys.path.insert(0, "/home/ylu/project/spateo-release/")
# import spateo
sys.path.insert(0, "/home/ylu/project/spateo-release/")
import spateo as st
from spateo.alignment.methods.morpho_class import *
from spateo.alignment.methods.utils import generate_label_transfer_dict
sys.path.insert(0, "/home/ylu/project")
from utils import *
from pathlib import Path
import numpy as np

%load_ext autoreload
%autoreload 2

2024-07-11 22:16:18.392958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 22:16:18.754802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-11 22:16:18.754850: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-11 22:16:20.152788: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [3]:
order = [
    155, 153,151,149,147,144,142,140, 138, 143,141,139, 137,135,133,130,131,
129,127,125,123,121,119,117,115,113,111,109,107,105,103, 101, 99, 97, 95, 93,
91,89,87,85,81,83,79,77,75,73,71,67,65,63,61,59,57,55,53,51,49,47,45,43,41,39,
37,33,31,29,27,25,28,30,32,34,36,38,40,42,44,48,50,52,54,56,58,60,62,
    64,66,68,72,74,76,78,80,82,84,88,90,92,94,96,98,100,102,104,106,108,
    110,112,114,116,118,120,122,124,136,134,132,128,126
]
data_folder = "/dm7100f/yifan/StereoSeq_macaque_processed"
files = [f'macaque_T{o}.h5ad' for o in order]

In [4]:
# data_folder = "./results/split_slices/"
results_folder = "./results/Spateo/"
figures_folder = "./results/figures/Spateo"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
## Perform the Spateo alignment
from tqdm import tqdm
import time
sampling_num = 20000
spatial_key = 'r_spatial'
key_added = 'align_spatial'
for i in tqdm(range(len(files)-1)):
    slice1 = ad.read_h5ad(os.path.join(data_folder, files[i]))
    slice2 = ad.read_h5ad(os.path.join(data_folder, files[i+1]))
    slice1.obsm[spatial_key] = slice1.obsm[spatial_key].astype(np.float64)
    slice2.obsm[spatial_key] = slice2.obsm[spatial_key].astype(np.float64)
    slice1 = slice1[~np.isnan(slice1.obsm[spatial_key].sum(1))]
    slice2 = slice2[~np.isnan(slice2.obsm[spatial_key].sum(1))]
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    top_genes_num = 5000
    top_genes_1 = np.argsort(-slice1.X.sum(0)).A[0][:top_genes_num] if slice1.shape[1] > top_genes_num else np.arange(slice1.shape[1])
    top_genes_2 = np.argsort(-slice2.X.sum(0)).A[0][:top_genes_num] if slice2.shape[1] > top_genes_num else np.arange(slice2.shape[1])
    slice1 = slice1[:,top_genes_1]
    slice2 = slice2[:,top_genes_2]
    
    time_start = time.time()
    adata = ad.concat([slice1,slice2], label='batch')
    PCA(adata, 'X', 'X_pca', 50, 'cuda')
    slice1.obsm['X_pca'] = adata[adata.obs['batch'] == '0'].obsm['X_pca'].copy()
    slice2.obsm['X_pca'] = adata[adata.obs['batch'] == '1'].obsm['X_pca'].copy()
    align_slices = [slice1, slice2]
    morpho_model = Morpho_pairwise(
        sampleA=align_slices[1],
        sampleB=align_slices[0],
        spatial_key=spatial_key,
        key_added=key_added,
        partial_robust_level=10,
        lambdaVF=1e1,
        beta=1e-1,
        sigma2_init_scale=0.1,
        device='1',
        sparse_calculation_mode=False,
        use_chunk=False,
        chunk_capacity=2,
        rep_layer=['X_pca'],
        rep_field=['obsm'],
        dissimilarity=['cos'],
        probability_type=['gauss'],
        nn_init=False,
    )
    P = morpho_model.run()
    align_slices[1].obsm[key_added] = morpho_model.optimal_RnA.copy()
    align_slices[0].obsm[key_added] = align_slices[0].obsm[spatial_key].copy()
    # align_slices[1].uns['align_spatial_iter'] = morpho_model.iter_added
    time_end = time.time()
    pi = P.T
    inlier = np.where(pi.sum(axis=1) > 0.1)[0]
    matches = [inlier, np.argmax(pi[inlier,:], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)